### Data Preprocessing

In [11]:
import boto3
import zipfile

# Define dataset path
s3_bucket = "id-classifier-images"
s3_key = "images.zip"
local_zip_path = "/tmp/images.zip"
dataset_dir = "/tmp/dataset"

# Download and extract dataset from S3
s3 = boto3.client("s3")
print(s3)
s3.download_file(s3_bucket, s3_key, local_zip_path)

with zipfile.ZipFile(local_zip_path, "r") as zip_ref:
    zip_ref.extractall(dataset_dir)

### Checking extracted dataset

import os

print("Checking /tmp/ contents:")
print(os.listdir("/tmp/"))  # List files/folders in /tmp

if os.path.exists(dataset_dir):
    print(f"Dataset directory exists: {dataset_dir}")
    print("Extracted files:")
    print(os.listdir(dataset_dir))  # List extracted files
else:
    print("Dataset directory NOT found. Extraction may have failed.")

### Define Image Transformations

In [17]:
# !pip install torchvision

In [16]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])


### Custom Dataset Class

In [19]:
import os
from PIL import Image
from torch.utils.data import DataLoader, Dataset

class IdentityDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dir))

        for class_idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(class_idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

dataset = IdentityDataset(dataset_dir, transform=transform)

NameError: name 'Dataset' is not defined